In [103]:
import nltk as nltk
import nltk.corpus  
from nltk.text import Text
from nltk.corpus import stopwords
import pandas as pd
import re
import sys
import matplotlib.pyplot as plt
import numpy as np
import scikitplot as skplt
import glob
import os
import spacy
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics
from sklearn import tree, svm
from sklearn.neural_network import MLPClassifier
from sklearn import utils

### Read in data

First section is only for the scored test set with generated summaries

In [41]:
# train dat
train_dta = pd.read_csv("Z:/final_data/cohorts_merged_training.csv",keep_default_na=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,69,325) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
train_dta.head()

,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death,encounter_id,enc_eio,visit_status_1,visit_status_2,...,technique,comparison,findings,impressions,elecsig,patient_raw,rawtext,encounter_id_diagnosed,date_diagnosed,ild_status
0,2068804,60,F,White,Not Hispanic or Latino,,7629913,I,Completed,,...,XR PORT CHEST 1V,None,Small lung volumes with bilateral airspace and...,Findings consistent with CHF/fluid overload.,**DATE<[**2017-09-06**]> 9:53 AM **NAME<EEE F...,7629913_446136,"XR PORT CHEST 1V, **DATE<[**2017-09-06**]> 9:1...",,,0
1,2068842,54,M,Black/African-American,Not Hispanic or Latino,,6781649,O,Completed,,...,,,,,,,,,,0
2,2068842,54,M,Black/African-American,Not Hispanic or Latino,,6781649,O,Completed,,...,Noncontrast thin section axial CT images of th...,None.,There is evidence of prior endoscopic surgery ...,1. Mild paranasal sinus disease including muco...,**DATE<[**2017-09-06**]> 1:36 PM **NAME<ZZZ AA...,6781649_1155203,"CT [**Company 1**] FUSION WO CONTRAST, **DATE<...",,,0
3,2068842,54,M,Black/African-American,Not Hispanic or Latino,,7649307,O,Completed,,...,,,,,,,,,,0
4,2068866,36,F,White,Hispanic or Latino,,7589589,O,Completed,,...,XR CHEST PA/LATERAL,None.,Right upper lobe resection via thoracotomy wit...,No significant abnormality.,**DATE<[**2017-08-21**]> 3:53 PM **NAME<EEE FF...,7589589_108353,"XR CHEST PA/LATERAL, **DATE<[**2017-08-21**]> ...",,,0


In [43]:
#Coalesce adm_date_d and start_date_d where no adm_date_d
train_dta['init_date'] = np.where(train_dta['adm_date_d']=='', train_dta['start_date_d'], train_dta['adm_date_d'])

In [44]:
# Set diagnosis at time of visit
train_dta['ild_diag_prior_to_visit'] = np.where((train_dta['init_date'] < train_dta['date_diagnosed']) | (
train_dta['date_diagnosed'] == ''), 0, 1)

### Let's get only the rows with an impression to build off of. 

We'll need to grab the rows where the ild_diag_prior_to_visit is 0, and has a "finding"

In [45]:
train_dta.head().loc[train_dta.findings.notnull()]

,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death,encounter_id,enc_eio,visit_status_1,visit_status_2,...,findings,impressions,elecsig,patient_raw,rawtext,encounter_id_diagnosed,date_diagnosed,ild_status,init_date,ild_diag_prior_to_visit
0,2068804,60,F,White,Not Hispanic or Latino,,7629913,I,Completed,,...,Small lung volumes with bilateral airspace and...,Findings consistent with CHF/fluid overload.,**DATE<[**2017-09-06**]> 9:53 AM **NAME<EEE F...,7629913_446136,"XR PORT CHEST 1V, **DATE<[**2017-09-06**]> 9:1...",,,0,2015-08-23,0
1,2068842,54,M,Black/African-American,Not Hispanic or Latino,,6781649,O,Completed,,...,,,,,,,,0,2015-09-30,0
2,2068842,54,M,Black/African-American,Not Hispanic or Latino,,6781649,O,Completed,,...,There is evidence of prior endoscopic surgery ...,1. Mild paranasal sinus disease including muco...,**DATE<[**2017-09-06**]> 1:36 PM **NAME<ZZZ AA...,6781649_1155203,"CT [**Company 1**] FUSION WO CONTRAST, **DATE<...",,,0,2015-09-30,0
3,2068842,54,M,Black/African-American,Not Hispanic or Latino,,7649307,O,Completed,,...,,,,,,,,0,2015-09-18,0
4,2068866,36,F,White,Hispanic or Latino,,7589589,O,Completed,,...,Right upper lobe resection via thoracotomy wit...,No significant abnormality.,**DATE<[**2017-08-21**]> 3:53 PM **NAME<EEE FF...,7589589_108353,"XR CHEST PA/LATERAL, **DATE<[**2017-08-21**]> ...",,,0,2015-03-31,0


In [46]:
train_dta_ild = train_dta[(train_dta.ild_diag_prior_to_visit==0) & (train_dta.findings.str.len()>0)]
train_dta_ild.shape

(357178, 343)

In [47]:
train_dta[(train_dta.ild_diag_prior_to_visit==0)].shape

(887297, 343)

### Spacy cleaning


In [48]:
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
stops = stopwords.words("english")

def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

train_dta_ild['findings_cleaned'] = train_dta_ild['findings'].apply(normalize,lowercase=True,remove_stopwords=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [49]:
train_dta_ild.to_csv('train_dta_ild.csv')

In [2]:
%time train_dta_ild = pd.read_csv('train_dta_ild.csv')

<string>:2: DtypeWarning: Columns (12,19,36,55,64,66,67,164,167,170,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210,212,213,215,216,218,219,221,222,224,225,227,228,230,231,233,234,236,237,239,240,242,243,245,246,248,249,251,252,254,255,257,258,260,261,263,264,266,267,269,270,272,273,275,276,278,279,281,282,284,285,287,288,290,291,293,294,296,297,299,300,302,303,305,306,340) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 47.8 s


In [51]:
train_dta_ild.head()

,Unnamed: 0,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death,encounter_id,enc_eio,visit_status_1,...,impressions,elecsig,patient_raw,rawtext,encounter_id_diagnosed,date_diagnosed,ild_status,init_date,ild_diag_prior_to_visit,findings_cleaned
0,0,2068804,60.0,F,White,Not Hispanic or Latino,NaN,7629913,I,Completed,...,Findings consistent with CHF/fluid overload.,**DATE<[**2017-09-06**]> 9:53 AM **NAME<EEE F...,7629913_446136,"XR PORT CHEST 1V, **DATE<[**2017-09-06**]> 9:1...",NaN,NaN,0,2015-08-23,0,small lung volume bilateral airspace interstit...
1,2,2068842,54.0,M,Black/African-American,Not Hispanic or Latino,NaN,6781649,O,Completed,...,1. Mild paranasal sinus disease including muco...,**DATE<[**2017-09-06**]> 1:36 PM **NAME<ZZZ AA...,6781649_1155203,"CT [**Company 1**] FUSION WO CONTRAST, **DATE<...",NaN,NaN,0,2015-09-30,0,evidence prior endoscopic surgery include leav...
2,4,2068866,36.0,F,White,Hispanic or Latino,NaN,7589589,O,Completed,...,No significant abnormality.,**DATE<[**2017-08-21**]> 3:53 PM **NAME<EEE FF...,7589589_108353,"XR CHEST PA/LATERAL, **DATE<[**2017-08-21**]> ...",NaN,NaN,0,2015-03-31,0,right upper lobe resection via thoracotomy cli...
3,6,2068882,28.0,M,Black/African-American,Not Hispanic or Latino,NaN,6168287,E,Completed,...,No significant abnormality.,**DATE<[**2017-09-05**]> 10:19 AM **NAME<GGG H...,6168287_509640,"XR CHEST PA/LATERAL, **DATE<[**2017-09-05**]> ...",NaN,NaN,0,2015-03-27,0,significant pleural parenchymal abnormality . ...
4,8,2068889,57.0,F,Black/African-American,Not Hispanic or Latino,NaN,6336463,E,Completed,...,No evidence of intracranial hemorrhage.,**DATE<[**2017-08-21**]> 8:16 AM **NAME<SSS TT...,6336463_568512,"CT HEAD WO, **DATE<[**2017-08-20**]> 10:25 PM ...",NaN,NaN,0,2015-04-07,0,intracranial hemorrhage identify . intracrania...


### Set up data for modeling

In [3]:
X = np.array(train_dta_ild['findings_cleaned'])
Y = np.array(train_dta_ild['ild_status'])
print(X.shape)
print(Y.shape)


(357178,)
(357178,)


In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state=1)
print(X_train.shape)
print(X_test.shape)


(267883,)
(89295,)


In [5]:
vect = CountVectorizer()
vect.fit(X_train)
%time X_train_dtm = vect.transform(X_train)


Wall time: 26.3 s


In [6]:
# test
X_test_dtm = vect.transform(np.array(X_test))
X_test_dtm


<89295x18726 sparse matrix of type '<class 'numpy.int64'>'
	with 4186426 stored elements in Compressed Sparse Row format>

### Classified construction - raw sample

In [56]:
classifier_names = ['NaiveBayes', 'Logit', 'DecisionTree', 'GradientBoosting', 'RandomForest', 'MLP']#, 'LogitRegularized', 'DecisionTreePruned']

# Simple Classifiers
nb_raw = MultinomialNB()
logit_raw = LogisticRegression(C=1e5)
dt_raw = tree.DecisionTreeClassifier(criterion="gini", min_samples_split=10)
gbt_raw = GradientBoostingClassifier(learning_rate=0.2, n_estimators=1000, max_depth=3)
rf_raw = RandomForestClassifier(max_depth = 3, random_state = 0)
mlp_raw = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(42,42,42), random_state = 1)

# Regularized Classifiers
# tuned_parameters = {'C': list(np.power(10.0, np.arange(-10, 10)))}
# parameters = {'max_depth':list(range(3,20))}

#logit_reg = GridSearchCV(LogisticRegression(), tuned_parameters, cv=3, scoring="roc_auc")
#dt_pruned = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4) 

classifier_obj_raw = [nb_raw, logit_raw, dt_raw, gbt_raw, rf_raw, mlp_raw]#, logit_reg, dt_pruned]

models_dict_raw = dict(zip(classifier_names, classifier_obj_raw))

### Calculating Results

In [11]:
def add_results(results, clf, classifier_name, x_train, x_test, y_train, y_test):
    results[classifier_name] = {'Accuracy': accuracy_score(y_test, clf.predict(x_test)),
                        'Test Error Rate': np.mean(clf.predict(x_test) != y_test),
                        'Training Error Rate': np.mean(clf.predict(x_train) != y_train),
                        'Classification Report': classification_report(y_test, clf.predict(x_test)),
                        'ConfMat': pd.crosstab(index=y_test, columns=clf.predict(x_test), rownames=['True'], colnames=['Predicted'])
                        }
    return results


def plot_roc_curve(clf, classifier_name, x_test, y_test):
    fpr, tpr, threshoolds = roc_curve(y_test, clf.predict(x_test))

    fpr_rand = tpr_rand = np.linspace(0, 1, 10)
    plt.plot(fpr, tpr) 
    plt.plot(fpr_rand, tpr_rand, linestyle='--') 
    plt.title('ROC Curve for %s' % classifier_name)
    plt.show()

Let's calculate our evaluation metrics for the classifiers we defined

In [ ]:
# on the data that has not been resampled
model_results_raw = {}

for k, v in models_dict_raw.items():
    print(f'Model: {k}')
    %time v.fit(X_train_dtm, Y_train)
    add_results(model_results_raw, v, k, X_train_dtm, X_test_dtm, Y_train, Y_test)

In [12]:
def describe_results(classifier_name, classifier_metric, model_results_dict):
    print("%s %s is:\n" % (classifier_name, classifier_metric), model_results_dict[classifier_name][classifier_metric])

### Raw sample results

In [60]:
for k,v in model_results_raw.items():
    print(f'Model: {k}')
    describe_results(k, 'Classification Report', model_results_raw)

Model: NaiveBayes
NaiveBayes Classification Report is:
               precision    recall  f1-score   support

           0       0.98      0.65      0.78     86667
           1       0.05      0.66      0.10      2628

   micro avg       0.65      0.65      0.65     89295
   macro avg       0.52      0.66      0.44     89295
weighted avg       0.96      0.65      0.76     89295

Model: Logit
Logit Classification Report is:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     86667
           1       0.28      0.05      0.08      2628

   micro avg       0.97      0.97      0.97     89295
   macro avg       0.62      0.52      0.53     89295
weighted avg       0.95      0.97      0.96     89295

Model: DecisionTree
DecisionTree Classification Report is:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98     86667
           1       0.22      0.11      0.15      2628

   micro avg       0

### TF-IDF

Run raw samples training data through TF-IDF transformer to get term and inverse document frequencies

In [7]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_dtm)
X_train_tfidf.shape

(267883, 18726)

In [8]:
tfidf_transformer = TfidfTransformer()
X_test_tfidf = tfidf_transformer.fit_transform(X_test_dtm)
X_test_tfidf.shape

(89295, 18726)

#### Classifiers

In [13]:
classifier_names = ['NaiveBayes', 'Logit', 'DecisionTree', 'GradientBoosting', 'RandomForest', 'MLP']#, 'LogitRegularized', 'DecisionTreePruned']

# Simple Classifiers
nb_tfidf = MultinomialNB()
logit_tfidf = LogisticRegression(C=1e5)
dt_tfidf = tree.DecisionTreeClassifier(criterion="gini", min_samples_split=10)
gbt_tfidf = GradientBoostingClassifier(learning_rate=0.2, n_estimators=1000, max_depth=3)
rf_tfidf = RandomForestClassifier(max_depth = 3, random_state = 0)
mlp_tfidf = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(42,42,42), random_state = 1)

# Regularized Classifiers
# tuned_parameters = {'C': list(np.power(10.0, np.arange(-10, 10)))}
# parameters = {'max_depth':list(range(3,20))}

#logit_reg = GridSearchCV(LogisticRegression(), tuned_parameters, cv=3, scoring="roc_auc")
#dt_pruned = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4) 

classifier_obj_tfidf = [nb_tfidf, logit_tfidf, dt_tfidf, gbt_tfidf, rf_tfidf, mlp_tfidf]#, logit_reg, dt_pruned]

models_dict_tfidf = dict(zip(classifier_names, classifier_obj_tfidf))

In [14]:
# on the data that has not been resampled
model_results_tfidf = {}

for k, v in models_dict_tfidf.items():
    print(f'Model: {k}')
    %time v.fit(X_train_tfidf, Y_train)
    add_results(model_results_tfidf, v, k, X_train_tfidf, X_test_tfidf, Y_train, Y_test)

Model: NaiveBayes
Wall time: 328 ms
Model: Logit


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 36min 20s
Model: DecisionTree
Wall time: 26min 26s
Model: GradientBoosting
Wall time: 2h 39min 38s
Model: RandomForest


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 2.03 s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Model: MLP
Wall time: 13min 35s


In [15]:
for k,v in model_results_tfidf.items():
    print(f'Model: {k}')
    describe_results(k, 'Classification Report', model_results_tfidf)

Model: NaiveBayes
NaiveBayes Classification Report is:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     86667
           1       0.05      0.00      0.00      2628

   micro avg       0.97      0.97      0.97     89295
   macro avg       0.51      0.50      0.49     89295
weighted avg       0.94      0.97      0.96     89295

Model: Logit
Logit Classification Report is:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     86667
           1       0.26      0.05      0.08      2628

   micro avg       0.97      0.97      0.97     89295
   macro avg       0.61      0.52      0.53     89295
weighted avg       0.95      0.97      0.96     89295

Model: DecisionTree
DecisionTree Classification Report is:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98     86667
           1       0.18      0.13      0.15      2628

   micro avg       0

## Class Sizes
 
Upsample minority class in training.

Later downsample.

In [16]:
len(train_dta_ild[(train_dta_ild['ild_status']==1)])/len(train_dta_ild)

0.028044840387705848

In [17]:
len(train_dta_ild[(train_dta_ild['ild_status']==0)])

347161

Split before resampling

In [18]:
%time X_train_b4_rs, X_test_b4_rs, Y_train_b4_rs, Y_test_b4_rs = train_test_split(train_dta_ild,train_dta_ild['ild_status'],random_state=1)

Wall time: 8.57 s


Out of curiosity, what is the class balance in training and test

In [19]:
print(f'train balance: {len(X_train_b4_rs[(X_train_b4_rs["ild_status"]==1)])/len(X_train_b4_rs)}')
print(f'test balance: {len(X_test_b4_rs[(X_test_b4_rs["ild_status"]==1)])/len(X_test_b4_rs)}')

train balance: 0.02758293732711669
test balance: 0.029430539223920714


In [20]:
len(X_test_b4_rs[(X_test_b4_rs["ild_status"]==1)])

2628

In [26]:
X_test_b4_rs

,Unnamed: 0,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death,encounter_id,enc_eio,visit_status_1,...,impressions,elecsig,patient_raw,rawtext,encounter_id_diagnosed,date_diagnosed,ild_status,init_date,ild_diag_prior_to_visit,findings_cleaned
200391,581097,1538469,80.0,F,White,Not Hispanic or Latino,NaN,5757545,O,Completed,...,1. No evidence of local recurrence or metastat...,**DATE<[**2016-07-03**]> 10:51 AM **NAME<III J...,5757545_134944,"CT CHEST ABDOMEN PELVIS W, **DATE<[**2016-07-0...",NaN,NaN,0,2014-08-29,0,chest : lung pleura : minimal biapical scar ev...
302499,872243,1946490,54.0,F,White,Not Hispanic or Latino,2013-05-29,4875850,O,Completed,...,1. No acute intracranial abnormality. 2. Stab...,**DATE<[**2015-04-19**]> 7:23 AM **NAME<SSS TT...,4875850_454861,"CT HEAD WO, **DATE<[**2015-04-18**]> 3:40 PM ...",NaN,NaN,0,2013-04-11,0,right frontal shunt catheter terminate within ...
296923,855571,1915514,60.0,M,Patient Declined,Patient Declined,NaN,8885202,O,Completed,...,No interval suspicious change or evidence of n...,**DATE<[**2018-11-11**]> 11:07 AM **NAME<VVV W...,8885202_419804,CT CHEST W **DATE<[**2018-11-11**]> 8:45 AM\r...,NaN,NaN,0,2017-01-05,0,lung pleura : grossly unchanged thicken along ...
73189,228458,173036,59.0,F,Black/African-American,Not Hispanic or Latino,NaN,5825838,O,Completed,...,Pulmonary sarcoidosis. No new opacity to sugg...,"**NAME<SSS TTT>, MD I personally reviewed the...",5825838_445942,"XR CHEST PA/LATERAL, **DATE<[**2017-07-13**]> ...",NaN,NaN,0,2015-03-21,0,unchanged bilateral upper lobe fibrosis compat...
50115,162127,52217,57.0,M,Black/African-American,Not Hispanic or Latino,NaN,9038976,I,No Show,...,No significant interval change in the acute he...,**DATE<[**2019-08-22**]> 1:22 PM **NAME<SSS TT...,9038976_767770,"CT HEAD WO, **DATE<[**2019-08-22**]> 1:06 PM ...",NaN,NaN,0,2016-05-17,0,redemonstration hyperattenuating hemorrhage wi...
14782,42294,2105221,55.0,M,White,Not Hispanic or Latino,NaN,6559171,O,Completed,...,Stable loculated right hydropneumothorax with ...,**DATE<[**2018-06-06**]> 9:23 AM **NAME<SSS TT...,6559171_637527,"XR CHEST PA/LATERAL, **DATE<[**2018-06-06**]> ...",NaN,NaN,0,2015-09-28,0,right lung opacity loculated pleural fluid air...
78530,243113,207979,34.0,F,Black/African-American,Not Hispanic or Latino,2017-06-30,9163867,I,No Show,...,Interstitial and airspace opacities now left m...,**DATE<[**2019-04-10**]> 11:24 AM **NAME<SSS T...,9163867_267565,"XR PORT CHEST 1V, **DATE<[**2019-04-10**]> 7:1...",NaN,NaN,0,2017-04-24,0,"unchanged leave - side opacity , improve right..."
106571,320098,542784,50.0,F,Black/African-American,Not Hispanic or Latino,NaN,6469229,NaN,Completed,...,"Cardiomegaly, and findings suggestive of pulmo...",**DATE<[**2016-01-08**]> 11:31 AM **NAME<HHH I...,6469229_512918,"XR PORT CHEST 1V, **DATE<[**2016-01-08**]> 10:...",NaN,NaN,0,2013-09-29,0,severe cardiomegaly large central pulmonary ar...
174790,509190,1327157,69.0,M,Unknown,Unknown,NaN,8655534,I,Completed,...,"Increasing left-sided atelectasis, now nearly ...",**DATE<[**2018-07-27**]> 5:16 PM **NAME<TTT UU...,8655534_267492,"XR PORT CHEST 1V, **DATE<[**2018-07-27**]> 5:1...",NaN,NaN,0,2016-07-13,0,increase leave - side atelectasis . abrupt cut...
72203,225838,167102,72.0,M,Black/African-American,Not Hispanic or Latino,NaN,6713683,O,Completed,...,1. Esophageal mass just above the level of the...,**DATE<[**2016-06-04**]> 2:01 PM [**Doctor Fir...,6713683_1178795,"CT CHEST AND UPPER ABD W, **DATE<[**2016-06-0...",NaN,NaN,0,2014-02-06,0,chest : lung pleura : suspicious pulmonary nod...


Now resample only from training.

### Make arrays again and vectorize

In [27]:
X_train_rs = np.array(X_train_b4_rs.findings_cleaned)
Y_train_rs = np.array(Y_train_b4_rs)
X_test_rs = np.array(X_test_b4_rs.findings_cleaned) # technically these were not resampled right
Y_test_rs = np.array(Y_test_b4_rs) # technically these were not resampled. is there any concern with having only 2% in our test?

In [28]:
vect_rs = CountVectorizer()
vect_rs.fit(X_train_rs)
%time X_train_dtm_rs = vect_rs.transform(X_train_rs)

Wall time: 25.7 s


In [29]:
X_train_dtm_rs.shape

(267883, 18726)

In [31]:
# test
X_test_dtm_rs = vect_rs.transform(np.array(X_test_rs))
X_test_dtm_rs


<89295x18726 sparse matrix of type '<class 'numpy.int64'>'
	with 4186426 stored elements in Compressed Sparse Row format>

In [32]:
sm = SMOTE(random_state=12, ratio = 1.0)
X_train_res, Y_train_res = sm.fit_sample(X_train_dtm_rs, Y_train_rs)

In [33]:
X_train_res.shape

(520988, 18726)

### Construct Classifers
All of the models selected are well suited for classification problems. We will run simple classifiers as well as some classifiers with tuned parameters to see which best handle our data.

In [34]:
classifier_names = ['NaiveBayes', 'Logit', 'DecisionTree', 'GradientBoosting', 'RandomForest', 'MLP']#, 'LogitRegularized', 'DecisionTreePruned']

# Simple Classifiers
nb_res = MultinomialNB()
logit_res = LogisticRegression(C=1e5)
dt_res = tree.DecisionTreeClassifier(criterion="gini", min_samples_split=10)
gbt_res = GradientBoostingClassifier(learning_rate=0.2, n_estimators=1000, max_depth=3)
rf_res = RandomForestClassifier(max_depth = 3, random_state = 0)
mlp_res = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(42,42,42), random_state = 1)

# Regularized Classifiers
# tuned_parameters = {'C': list(np.power(10.0, np.arange(-10, 10)))}
# parameters = {'max_depth':list(range(3,20))}

#logit_reg = GridSearchCV(LogisticRegression(), tuned_parameters, cv=3, scoring="roc_auc")
#dt_pruned = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4) 

classifier_obj_res = [nb_res, logit_res, dt_res, gbt_res, rf_res, mlp_res]#, logit_reg, dt_pruned]

models_dict_res = dict(zip(classifier_names, classifier_obj_res))

### Model fitting

In [35]:
# on the data that has been resampled
model_results_res = {}

for k, v in models_dict_res.items():
    print(f'Model: {k}')
    %time v.fit(X_train_res, Y_train_res)
    add_results(model_results_res, v, k, X_train_res, X_test_dtm_rs, Y_train_res, Y_test_rs)

Model: NaiveBayes
Wall time: 703 ms
Model: Logit


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Wall time: 30min 45s
Model: DecisionTree
Wall time: 21min 21s
Model: GradientBoosting
Wall time: 1h 57min 11s
Model: RandomForest


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 4.25 s
Model: MLP
Wall time: 28min 23s


### Evaluation metrics.

In [36]:
for k,v in model_results_res.items():
    print(f'Model: {k}')
    describe_results(k, 'Classification Report', model_results_res)
    print(f'Model Accuracy: {k}')
    describe_results(k, 'Accuracy', model_results_res)

Model: NaiveBayes
NaiveBayes Classification Report is:
               precision    recall  f1-score   support

           0       0.98      0.60      0.74     86667
           1       0.04      0.55      0.07      2628

   micro avg       0.60      0.60      0.60     89295
   macro avg       0.51      0.57      0.41     89295
weighted avg       0.95      0.60      0.72     89295

Model Accuracy: NaiveBayes
NaiveBayes Accuracy is:
 0.5974242678761409
Model: Logit
Logit Classification Report is:
               precision    recall  f1-score   support

           0       0.97      0.90      0.94     86667
           1       0.06      0.20      0.09      2628

   micro avg       0.88      0.88      0.88     89295
   macro avg       0.52      0.55      0.51     89295
weighted avg       0.95      0.88      0.91     89295

Model Accuracy: Logit
Logit Accuracy is:
 0.8799708830281651
Model: DecisionTree
DecisionTree Classification Report is:
               precision    recall  f1-score   suppor

### Downsampling

In [37]:
rus = RandomUnderSampler(random_state=777)
%time X_train_RUS, Y_train_RUS = rus.fit_sample(X_train_tfidf, Y_train)

Wall time: 156 ms


#### Classifiers

In [38]:
classifier_names = ['NaiveBayes', 'Logit', 'DecisionTree', 'GradientBoosting', 'RandomForest', 'MLP']#, 'LogitRegularized', 'DecisionTreePruned']

# Simple Classifiers
nb_ds = MultinomialNB()
logit_ds = LogisticRegression(C=1e5)
dt_ds = tree.DecisionTreeClassifier(criterion="gini", min_samples_split=10)
gbt_ds = GradientBoostingClassifier(learning_rate=0.2, n_estimators=1000, max_depth=3)
rf_ds = RandomForestClassifier(max_depth = 3, random_state = 0)
mlp_ds = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(42,42,42), random_state = 1)

# Regularized Classifiers
# tuned_parameters = {'C': list(np.power(10.0, np.arange(-10, 10)))}
# parameters = {'max_depth':list(range(3,20))}

#logit_reg = GridSearchCV(LogisticRegression(), tuned_parameters, cv=3, scoring="roc_auc")
#dt_pruned = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4) 

classifier_obj_ds = [nb_ds, logit_ds, dt_ds, gbt_ds, rf_ds, mlp_ds]#, logit_reg, dt_pruned]

models_dict_ds = dict(zip(classifier_names, classifier_obj_ds))

In [39]:
# on the data that has not been resampled
model_results_ds = {}

for k, v in models_dict_ds.items():
    print(f'Model: {k}')
    %time v.fit(X_train_RUS, Y_train_RUS)
    add_results(model_results_ds, v, k, X_train_RUS, X_test_tfidf, Y_train_RUS, Y_test)

Model: NaiveBayes
Wall time: 15.6 ms
Model: Logit


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Wall time: 15.3 s
Model: DecisionTree
Wall time: 8.21 s
Model: GradientBoosting
Wall time: 5min 56s
Model: RandomForest


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 125 ms
Model: MLP
Wall time: 1min 20s


In [40]:
for k,v in model_results_ds.items():
    print(f'Model: {k}')
    describe_results(k, 'Classification Report', model_results_ds)

Model: NaiveBayes
NaiveBayes Classification Report is:
               precision    recall  f1-score   support

           0       0.99      0.53      0.69     86667
           1       0.05      0.76      0.09      2628

   micro avg       0.54      0.54      0.54     89295
   macro avg       0.52      0.65      0.39     89295
weighted avg       0.96      0.54      0.67     89295

Model: Logit
Logit Classification Report is:
               precision    recall  f1-score   support

           0       0.98      0.63      0.77     86667
           1       0.05      0.67      0.10      2628

   micro avg       0.63      0.63      0.63     89295
   macro avg       0.52      0.65      0.43     89295
weighted avg       0.96      0.63      0.75     89295

Model: DecisionTree
DecisionTree Classification Report is:
               precision    recall  f1-score   support

           0       0.98      0.61      0.75     86667
           1       0.05      0.63      0.09      2628

   micro avg       0

In [61]:
for k,v in model_results_ds.items():
    print(f'Model: {k}')
    describe_results(k, 'ConfMat', model_results_ds)

Model: NaiveBayes
NaiveBayes ConfMat is:
 Predicted      0      1
True                   
0          46146  40521
1            629   1999
Model: Logit
Logit ConfMat is:
 Predicted      0      1
True                   
0          54504  32163
1            862   1766
Model: DecisionTree
DecisionTree ConfMat is:
 Predicted      0      1
True                   
0          52976  33691
1            965   1663
Model: GradientBoosting
GradientBoosting ConfMat is:
 Predicted      0      1
True                   
0          59729  26938
1            794   1834
Model: RandomForest
RandomForest ConfMat is:
 Predicted      0      1
True                   
0          39753  46914
1            675   1953
Model: MLP
MLP ConfMat is:
 Predicted      0      1
True                   
0          60345  26322
1            828   1800


In [62]:
for k,v in model_results_ds.items():
    print(f'Model: {k}')
    describe_results(k, 'Accuracy', model_results_ds)

Model: NaiveBayes
NaiveBayes Accuracy is:
 0.5391679265356403
Model: Logit
Logit Accuracy is:
 0.630158463519794
Model: DecisionTree
DecisionTree Accuracy is:
 0.6118931631110365
Model: GradientBoosting
GradientBoosting Accuracy is:
 0.689433898874517
Model: RandomForest
RandomForest Accuracy is:
 0.4670586259029061
Model: MLP
MLP Accuracy is:
 0.6959516210314127


### Adding SVM

In [97]:
svm_ds = svm.SVC(gamma='scale')

In [98]:
%time svm_ds.fit(X_train_RUS, Y_train_RUS)

Wall time: 2min 18s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [99]:
%time add_results(model_results_ds, svm_ds, 'SVM', X_train_RUS, X_test_tfidf, Y_train_RUS, Y_test)

Wall time: 54min 14s


{'DecisionTree': {'Accuracy': 0.6118931631110365,
  'Classification Report': '              precision    recall  f1-score   support\n\n           0       0.98      0.61      0.75     86667\n           1       0.05      0.63      0.09      2628\n\n   micro avg       0.61      0.61      0.61     89295\n   macro avg       0.51      0.62      0.42     89295\nweighted avg       0.95      0.61      0.73     89295\n',
  'ConfMat': Predicted      0      1
  True                   
  0          52976  33691
  1            965   1663,
  'Test Error Rate': 0.3881068368889635,
  'Training Error Rate': 0.041209906617945595},
 'GradientBoosting': {'Accuracy': 0.689433898874517,
  'Classification Report': '              precision    recall  f1-score   support\n\n           0       0.99      0.69      0.81     86667\n           1       0.06      0.70      0.12      2628\n\n   micro avg       0.69      0.69      0.69     89295\n   macro avg       0.53      0.69      0.46     89295\nweighted avg       0

In [100]:
describe_results('SVM', 'Accuracy', model_results_ds)

SVM Accuracy is:
 0.5640069432778991


In [101]:
describe_results('SVM', 'ConfMat', model_results_ds)

SVM ConfMat is:
 Predicted      0      1
True                   
0          48409  38258
1            674   1954


In [ ]:
def add_results(results, clf, classifier_name, x_train, x_test, y_train, y_test):
    results[classifier_name] = {'Accuracy': accuracy_score(y_test, clf.predict(x_test)),
                        'Test Error Rate': np.mean(clf.predict(x_test) != y_test),
                        'Training Error Rate': np.mean(clf.predict(x_train) != y_train),
                        'Classification Report': classification_report(y_test, clf.predict(x_test)),
                        'ConfMat': pd.crosstab(index=y_test, columns=clf.predict(x_test), rownames=['True'], colnames=['Predicted'])
                        }
    return results


In [105]:
# on the data that has not been resampled
model_results_ds = {}

for k, v in models_dict_ds.items():
    print(f'Model: {k}')
    %time v.fit(X_train_RUS, Y_train_RUS)
    add_results(model_results_ds, v, k, X_train_RUS, X_test_tfidf, Y_train_RUS, Y_test)

{'DecisionTree': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=10,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'),
 'GradientBoosting': GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.2, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=1000,
               n_iter_no_change=None, presort='auto', random_state=None,
               subsample=1.0, tol=0.0001, validation_fraction=0.1,
               verbose=0, warm_start=False),
 'Logit': LogisticRegression(C=100000.0, class_wei

In [104]:
models_dict_ds['SVM'] = svm_ds

### Adding F1 Score to results dict

In [115]:
for k,v in models_dict_ds.items():
    print(f'Model: {k}')
    %time model_results_ds[k]['F1Score'] = f1_score(Y_test, v.predict(X_test_tfidf))

Model: NaiveBayes
Wall time: 93.8 ms
Model: Logit
Wall time: 62.4 ms
Model: DecisionTree
Wall time: 125 ms
Model: GradientBoosting
Wall time: 3.83 s
Model: RandomForest
Wall time: 407 ms
Model: MLP
Wall time: 547 ms
Model: SVM
Wall time: 12min 34s


In [116]:
model_results_ds['SVM']

{'Accuracy': 0.5640069432778991,
 'Classification Report': '              precision    recall  f1-score   support\n\n           0       0.99      0.56      0.71     86667\n           1       0.05      0.74      0.09      2628\n\n   micro avg       0.56      0.56      0.56     89295\n   macro avg       0.52      0.65      0.40     89295\nweighted avg       0.96      0.56      0.69     89295\n',
 'ConfMat': Predicted      0      1
 True                   
 0          48409  38258
 1            674   1954,
 'F1Score': 0.09122315592903829,
 'Test Error Rate': 0.4359930567221009,
 'Training Error Rate': 0.34659629178508594}

In [119]:
for k,v in model_results_ds.items():
    print(f'Model: {k} F1 Score')
    print(model_results_ds[k]['F1Score'])

Model: NaiveBayes F1 Score
0.08855320279968104
Model: Logit F1 Score
0.09661624312717126
Model: DecisionTree F1 Score
0.08756779527144437
Model: GradientBoosting F1 Score
0.11681528662420383
Model: RandomForest F1 Score
0.075852024468395
Model: MLP F1 Score
0.11707317073170732
Model: SVM F1 Score
0.09122315592903829


In [120]:
for k,v in models_dict_ds.items():
    print(f'Model: {k}')
    print(model_results_ds[k]['Classification Report'])
    #%time model_results_ds[k]['F1Score'] = f1_score(Y_test, v.predict(X_test_tfidf))

Model: NaiveBayes
              precision    recall  f1-score   support

           0       0.99      0.53      0.69     86667
           1       0.05      0.76      0.09      2628

   micro avg       0.54      0.54      0.54     89295
   macro avg       0.52      0.65      0.39     89295
weighted avg       0.96      0.54      0.67     89295

Model: Logit
              precision    recall  f1-score   support

           0       0.98      0.63      0.77     86667
           1       0.05      0.67      0.10      2628

   micro avg       0.63      0.63      0.63     89295
   macro avg       0.52      0.65      0.43     89295
weighted avg       0.96      0.63      0.75     89295

Model: DecisionTree
              precision    recall  f1-score   support

           0       0.98      0.61      0.75     86667
           1       0.05      0.63      0.09      2628

   micro avg       0.61      0.61      0.61     89295
   macro avg       0.51      0.62      0.42     89295
weighted avg       0.95

### Grouping findings by patient

In [41]:
data = train_dta_ild[['patient_id','findings_cleaned', 'ild_status']]

In [42]:
%time all_findings_data = pd.DataFrame({'all_findings_cleaned' : data.groupby(['patient_id','ild_status'])['findings_cleaned'].apply(lambda x: ''.join(x))}).reset_index()

Wall time: 8.71 s


In [43]:
all_findings_data.head()

,patient_id,ild_status,all_findings_cleaned
0,99,0,lung nodule previously visualize right lateral...
1,104,0,unenhanced head ct : detectable acute intracra...
2,149,0,chronic pulmonary opacity possibly partly calc...
3,195,0,stable mild prominence mediastinum . cardiac /...
4,208,0,unremarkable cardiac mediastinal silhouette . ...


In [44]:
len(all_findings_data)

82695

In [45]:
len(data['patient_id'].unique())

82695

In [46]:
%time all_findings_data.to_csv('all_findings_data.csv')

Wall time: 18.2 s


### Set up data for modeling

In [47]:
X_patient = np.array(all_findings_data['all_findings_cleaned'])
Y_patient = np.array(all_findings_data['ild_status'])
print(X_patient.shape)
print(Y_patient.shape)


(82695,)
(82695,)


In [48]:
X_train_patient, X_test_patient, Y_train_patient, Y_test_patient = train_test_split(X_patient,Y_patient,random_state=1)
print(X_train_patient.shape)
print(X_test_patient.shape)


(62021,)
(20674,)


In [49]:
vect = CountVectorizer()
vect.fit(X_train_patient)
%time X_train_dtm_patient = vect.transform(X_train_patient)


Wall time: 21.5 s


In [50]:
# test
X_test_dtm_patient = vect.transform(np.array(X_test_patient))
X_test_dtm_patient


<20674x23557 sparse matrix of type '<class 'numpy.int64'>'
	with 2296712 stored elements in Compressed Sparse Row format>

### TF-IDF

Run patient samples training data through TF-IDF transformer to get term and inverse document frequencies

In [51]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf_patient = tfidf_transformer.fit_transform(X_train_dtm_patient)
X_train_tfidf_patient.shape

(62021, 23557)

In [52]:
tfidf_transformer = TfidfTransformer()
X_test_tfidf_patient = tfidf_transformer.fit_transform(X_test_dtm_patient)
X_test_tfidf_patient.shape

(20674, 23557)

### Classified construction - patient level sample - tfidf

In [53]:
classifier_names = ['NaiveBayes', 'Logit', 'DecisionTree', 'GradientBoosting', 'RandomForest', 'MLP', 'SVM']#, 'LogitRegularized', 'DecisionTreePruned']

# Simple Classifiers
nb_patient = MultinomialNB()
logit_patient = LogisticRegression(C=1e5)
dt_patient = tree.DecisionTreeClassifier(criterion="gini", min_samples_split=10)
gbt_patient = GradientBoostingClassifier(learning_rate=0.2, n_estimators=1000, max_depth=3)
rf_patient = RandomForestClassifier(max_depth = 3, random_state = 0)
mlp_patient = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(42,42,42), random_state = 1)
svm_patient = svm.SVC(gamma='scale')

# Regularized Classifiers
# tuned_parameters = {'C': list(np.power(10.0, np.arange(-10, 10)))}
# parameters = {'max_depth':list(range(3,20))}

#logit_reg = GridSearchCV(LogisticRegression(), tuned_parameters, cv=3, scoring="roc_auc")
#dt_pruned = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4) 

classifier_obj_patient = [nb_patient, logit_patient, dt_patient, gbt_patient, rf_patient, mlp_patient, svm_patient]#, logit_reg, dt_pruned]

models_dict_patient = dict(zip(classifier_names, classifier_obj_patient))

In [54]:
# on the data that has not been resampled
model_results_patient = {}

for k, v in models_dict_patient.items():
    print(f'Model: {k}')
    %time v.fit(X_train_tfidf_patient, Y_train_patient)
    add_results(model_results_patient, v, k, X_train_tfidf_patient, X_test_tfidf_patient, Y_train_patient, Y_test_patient)

Model: NaiveBayes
Wall time: 125 ms
Model: Logit


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 10min 54s
Model: DecisionTree
Wall time: 7min 7s
Model: GradientBoosting
Wall time: 1h 5min 28s
Model: RandomForest


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 766 ms


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Model: MLP
Wall time: 5min 14s
Model: SVM
Wall time: 25min 51s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [55]:
for k,v in model_results_patient.items():
    print(f'Model: {k}')
    describe_results(k, 'Classification Report', model_results_patient)

Model: NaiveBayes
NaiveBayes Classification Report is:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99     20133
           1       0.00      0.00      0.00       541

   micro avg       0.97      0.97      0.97     20674
   macro avg       0.49      0.50      0.49     20674
weighted avg       0.95      0.97      0.96     20674

Model: Logit
Logit Classification Report is:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97     20133
           1       0.11      0.19      0.14       541

   micro avg       0.94      0.94      0.94     20674
   macro avg       0.54      0.58      0.55     20674
weighted avg       0.96      0.94      0.95     20674

Model: DecisionTree
DecisionTree Classification Report is:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98     20133
           1       0.18      0.15      0.16       541

   micro avg       0

In [56]:
model_results_patient['GradientBoosting']

{'Accuracy': 0.9657540872593596,
 'Classification Report': '              precision    recall  f1-score   support\n\n           0       0.98      0.99      0.98     20133\n           1       0.20      0.10      0.13       541\n\n   micro avg       0.97      0.97      0.97     20674\n   macro avg       0.59      0.55      0.56     20674\nweighted avg       0.96      0.97      0.96     20674\n',
 'ConfMat': Predicted      0    1
 True                 
 0          19911  222
 1            486   55,
 'Test Error Rate': 0.03424591274064042,
 'Training Error Rate': 0.00722335982973509}

### Classifiers - Patient Level Count Vectorized

In [57]:
classifier_names = ['NaiveBayes', 'Logit', 'DecisionTree', 'GradientBoosting', 'RandomForest', 'MLP', 'SVM']#, 'LogitRegularized', 'DecisionTreePruned']

# Simple Classifiers
nb_patient_ct = MultinomialNB()
logit_patient_ct = LogisticRegression(C=1e5)
dt_patient_ct = tree.DecisionTreeClassifier(criterion="gini", min_samples_split=10)
gbt_patient_ct = GradientBoostingClassifier(learning_rate=0.2, n_estimators=1000, max_depth=3)
rf_patient_ct = RandomForestClassifier(max_depth = 3, random_state = 0)
mlp_patient_ct = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(42,42,42), random_state = 1)
svm_patient_ct = svm.SVC(gamma='scale')

# Regularized Classifiers
# tuned_parameters = {'C': list(np.power(10.0, np.arange(-10, 10)))}
# parameters = {'max_depth':list(range(3,20))}

#logit_reg = GridSearchCV(LogisticRegression(), tuned_parameters, cv=3, scoring="roc_auc")
#dt_pruned = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4) 

classifier_obj_patient_ct = [nb_patient_ct, logit_patient_ct, dt_patient_ct, gbt_patient_ct, rf_patient_ct,
                     mlp_patient_ct, svm_patient_ct]#, logit_reg, dt_pruned]

models_dict_patient_ct = dict(zip(classifier_names, classifier_obj_patient_ct))

In [58]:
# on the data that has not been resampled
model_results_patient_ct = {}

for k, v in models_dict_patient_ct.items():
    print(f'Model: {k}')
    %time v.fit(X_train_dtm_patient, Y_train_patient)
    add_results(model_results_patient_ct, v, k, X_train_dtm_patient, X_test_dtm_patient, Y_train_patient, Y_test_patient)

Model: NaiveBayes
Wall time: 172 ms
Model: Logit


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Wall time: 3min 5s
Model: DecisionTree
Wall time: 1min 32s
Model: GradientBoosting
Wall time: 24min 35s
Model: RandomForest


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 703 ms


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Model: MLP
Wall time: 5min 41s
Model: SVM
Wall time: 58min 44s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [59]:
for k,v in model_results_patient_ct.items():
    print(f'Model: {k}')
    describe_results(k, 'Classification Report', model_results_patient_ct)

Model: NaiveBayes
NaiveBayes Classification Report is:
               precision    recall  f1-score   support

           0       0.99      0.82      0.90     20133
           1       0.08      0.55      0.14       541

   micro avg       0.82      0.82      0.82     20674
   macro avg       0.53      0.69      0.52     20674
weighted avg       0.96      0.82      0.88     20674

Model: Logit
Logit Classification Report is:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98     20133
           1       0.26      0.11      0.15       541

   micro avg       0.97      0.97      0.97     20674
   macro avg       0.62      0.55      0.57     20674
weighted avg       0.96      0.97      0.96     20674

Model: DecisionTree
DecisionTree Classification Report is:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98     20133
           1       0.15      0.10      0.12       541

   micro avg       0

In [68]:
for k,v in model_results_patient_ct.items():
    print(f'Model: {k}')
    describe_results(k, 'Accuracy', model_results_patient_ct)

Model: NaiveBayes
NaiveBayes Accuracy is:
 0.8154687046531875
Model: Logit
Logit Accuracy is:
 0.9682693237883332
Model: DecisionTree
DecisionTree Accuracy is:
 0.9610138338009093
Model: GradientBoosting
GradientBoosting Accuracy is:
 0.9684628035213312
Model: RandomForest
RandomForest Accuracy is:
 0.9738318661120248
Model: MLP
MLP Accuracy is:
 0.9718970687820451
Model: SVM
SVM Accuracy is:
 0.9738318661120248


### Downsampling Patient Level

In [63]:
rus = RandomUnderSampler(random_state=777)
%time X_train_RUS_patient, Y_train_RUS_patient = rus.fit_sample(X_train_dtm_patient, Y_train_patient)

Wall time: 36.3 ms


In [77]:
classifier_names = ['NaiveBayes', 'Logit', 'DecisionTree', 'GradientBoosting', 'RandomForest', 'MLP', 'SVM']#, 'LogitRegularized', 'DecisionTreePruned']

# Simple Classifiers
nb_patient_ct_RUS = MultinomialNB()
logit_patient_ct_RUS = LogisticRegression(C=1e5)
dt_patient_ct_RUS = tree.DecisionTreeClassifier(criterion="gini", min_samples_split=10)
gbt_patient_ct_RUS = GradientBoostingClassifier(learning_rate=0.2, n_estimators=1000, max_depth=3)
rf_patient_ct_RUS = RandomForestClassifier(max_depth = 3, random_state = 0)
mlp_patient_ct_RUS = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(42,42,42), random_state = 1)
svm_patient_ct_RUS = svm.SVC(gamma='scale', probability=True)

# Regularized Classifiers
# tuned_parameters = {'C': list(np.power(10.0, np.arange(-10, 10)))}
# parameters = {'max_depth':list(range(3,20))}

#logit_reg = GridSearchCV(LogisticRegression(), tuned_parameters, cv=3, scoring="roc_auc")
#dt_pruned = GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=4) 

classifier_obj_patient_ct_RUS = [nb_patient_ct_RUS, logit_patient_ct_RUS, dt_patient_ct_RUS,
                                 gbt_patient_ct_RUS, rf_patient_ct_RUS, mlp_patient_ct_RUS,
                                 svm_patient_ct_RUS]#, logit_reg, dt_pruned]

models_dict_patient_ct_RUS = dict(zip(classifier_names, classifier_obj_patient_ct_RUS))

In [78]:
# on the data that has been downsampled
model_results_patient_ct_RUS = {}

for k, v in models_dict_patient_ct_RUS.items():
    print(f'Model: {k}')
    %time v.fit(X_train_RUS_patient, Y_train_RUS_patient)
    add_results(model_results_patient_ct_RUS, v, k, X_train_RUS_patient, X_test_dtm_patient, Y_train_RUS_patient, Y_test_patient)

Model: NaiveBayes
Wall time: 9.52 ms
Model: Logit


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Wall time: 9.44 s
Model: DecisionTree
Wall time: 1.83 s
Model: GradientBoosting
Wall time: 1min 53s
Model: RandomForest


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 62.5 ms
Model: MLP
Wall time: 7min 59s
Model: SVM
Wall time: 1min 2s


In [79]:
for k,v in model_results_patient_ct_RUS.items():
    print(f'Model: {k}')
    describe_results(k, 'Classification Report', model_results_patient_ct_RUS)

Model: NaiveBayes
NaiveBayes Classification Report is:
               precision    recall  f1-score   support

           0       0.99      0.64      0.78     20133
           1       0.05      0.71      0.09       541

   micro avg       0.64      0.64      0.64     20674
   macro avg       0.52      0.67      0.43     20674
weighted avg       0.96      0.64      0.76     20674

Model: Logit
Logit Classification Report is:
               precision    recall  f1-score   support

           0       0.99      0.71      0.83     20133
           1       0.06      0.68      0.11       541

   micro avg       0.71      0.71      0.71     20674
   macro avg       0.52      0.69      0.47     20674
weighted avg       0.96      0.71      0.81     20674

Model: DecisionTree
DecisionTree Classification Report is:
               precision    recall  f1-score   support

           0       0.99      0.70      0.82     20133
           1       0.06      0.66      0.10       541

   micro avg       0

In [80]:
for k,v in model_results_patient_ct_RUS.items():
    print(f'Model: {k}')
    describe_results(k, 'Accuracy', model_results_patient_ct_RUS)

Model: NaiveBayes
NaiveBayes Accuracy is:
 0.6397891070910322
Model: Logit
Logit Accuracy is:
 0.7100706201025443
Model: DecisionTree
DecisionTree Accuracy is:
 0.6952210505949502
Model: GradientBoosting
GradientBoosting Accuracy is:
 0.7599883912160201
Model: RandomForest
RandomForest Accuracy is:
 0.6701170552384638
Model: MLP
MLP Accuracy is:
 0.7048466673115991
Model: SVM
SVM Accuracy is:
 0.8382509432136984


In [121]:
for k,v in model_results_patient_ct_RUS.items():
    print(f'Model: {k}')
    describe_results(k, 'Classification Report', model_results_patient_ct_RUS)

Model: NaiveBayes
NaiveBayes Classification Report is:
               precision    recall  f1-score   support

           0       0.99      0.64      0.78     20133
           1       0.05      0.71      0.09       541

   micro avg       0.64      0.64      0.64     20674
   macro avg       0.52      0.67      0.43     20674
weighted avg       0.96      0.64      0.76     20674

Model: Logit
Logit Classification Report is:
               precision    recall  f1-score   support

           0       0.99      0.71      0.83     20133
           1       0.06      0.68      0.11       541

   micro avg       0.71      0.71      0.71     20674
   macro avg       0.52      0.69      0.47     20674
weighted avg       0.96      0.71      0.81     20674

Model: DecisionTree
DecisionTree Classification Report is:
               precision    recall  f1-score   support

           0       0.99      0.70      0.82     20133
           1       0.06      0.66      0.10       541

   micro avg       0

In [81]:
model_results_patient_ct_RUS['SVM']

{'Accuracy': 0.8382509432136984,
 'Classification Report': '              precision    recall  f1-score   support\n\n           0       0.99      0.85      0.91     20133\n           1       0.08      0.52      0.14       541\n\n   micro avg       0.84      0.84      0.84     20674\n   macro avg       0.53      0.68      0.53     20674\nweighted avg       0.96      0.84      0.89     20674\n',
 'ConfMat': Predicted      0     1
 True                  
 0          17047  3086
 1            258   283,
 'Test Error Rate': 0.16174905678630164,
 'Training Error Rate': 0.30156815440289503}

In [93]:
describe_results('SVM', 'Classification Report', model_results_patient_ct_RUS)

SVM Classification Report is:
               precision    recall  f1-score   support

           0       0.99      0.85      0.91     20133
           1       0.08      0.52      0.14       541

   micro avg       0.84      0.84      0.84     20674
   macro avg       0.53      0.68      0.53     20674
weighted avg       0.96      0.84      0.89     20674



In [95]:
describe_results('SVM', 'ConfMat', model_results_patient_ct_RUS)

SVM ConfMat is:
 Predicted      0     1
True                  
0          17047  3086
1            258   283


### Export Probabilities of ILD by patient

In [82]:
def dt_preds(x):
    return svm_patient_ct_RUS.predict_proba(vect.transform(np.array([x])))[:,1][0]


In [83]:
% time all_findings_data['ild_prob'] = all_findings_data['all_findings_cleaned'].apply(dt_preds)

Wall time: 9min 30s


In [84]:
all_findings_data.head()

,patient_id,ild_status,all_findings_cleaned,ild_prob
0,99,0,lung nodule previously visualize right lateral...,0.674523
1,104,0,unenhanced head ct : detectable acute intracra...,0.455515
2,149,0,chronic pulmonary opacity possibly partly calc...,0.474806
3,195,0,stable mild prominence mediastinum . cardiac /...,0.574298
4,208,0,unremarkable cardiac mediastinal silhouette . ...,0.363837


In [91]:
all_findings_data.shape

(82695, 4)

In [87]:
%time all_findings_data[['patient_id', 'ild_status', 'ild_prob']].to_csv('ild_probability_by_patient.csv')

Wall time: 976 ms


In [90]:
len(train_dta_ild['patient_id'].unique())

82695